# Reference Forecasts

This [Jupyter notebook](https://jupyter.org) is designed to introduce you to the [Solar Forecast Arbiter's](https://solarforecastarbiter.org/) built-in reference forecast capabilities. It is divided into 2 sections:

1. [NWP-based forecasts](#NWP-based-forecasts)
2. [Persistence forecasts](#persistence-forecasts)

The API documentation is available [here](https://solarforecastarbiter-core.readthedocs.io/en/latest/reference_forecasts.html).

In [32]:
import datetime
from functools import partial
from pathlib import Path

import numpy as np
import pandas as pd

from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.palettes import Dark2_5 as PALETTE
TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select"
output_notebook()

Loading BokehJS ...

In [10]:
from solarforecastarbiter import datamodel

## NWP-based forecasts

Forecasts based on NWP model data are used for intraday and longer forecasts. The Solar Forecast Arbiter contains a set of functions to process data from NWP forecasts. Here, we explore some of the functionality with an emphasis on obtaining results. See the [NWP section of the documentation](https://solarforecastarbiter-core.readthedocs.io/en/latest/reference_forecasts.html#nwp) for additional information.

The `solarforecastarbiter-core` package includes a handful of subsetted NWP model runs for testing. We'll use these for our demonstration below. The models were all initialized at 2019-05-15 00Z and the subsets include about half a degree of latitude and longitude near Tucson, AZ.

In [11]:
from solarforecastarbiter.io import nwp
# find the files
base_path = Path(nwp.__file__).resolve().parents[0] / 'tests/data'
# define file loading function that knows where to find the files
load_forecast = partial(nwp.load_forecast, base_path=base_path)

In [20]:
# define coordinates
latitude = 32.2
longitude = -110.9
elevation = 700

# define initialization time, forecast start time, forecast end time
init_time = pd.Timestamp('20190515T0000Z')
start = pd.Timestamp('20190515T0100Z')
end = pd.Timestamp('20190518T0000Z')

In [34]:
from solarforecastarbiter.reference_forecasts import main, models

In [33]:
site = datamodel.Site(
    name='Tucson, AZ',
    latitude=32.2,
    longitude=-110.9,
    elevation=700,
    timezone='America/Phoenix'
)

In [43]:
# select the model
model = models.hrrr_subhourly_to_subhourly_instantaneous

# tell the model where to find the NWP data
model_wrapped = partial(model, load_forecast=load_forecast)

# load and process the NWP data
ghi, dni, dhi, air_temperature, wind_speed, ac_power = main.run(site, model_wrapped, init_time, start, end)

In [42]:
fig = figure(title="Irradiance (W/m^2)", tools=TOOLS, x_axis_type="datetime", plot_width=800, plot_height=400)
palette = iter(PALETTE)
for irrad, name in zip((ghi, dni, dhi), ('ghi', 'dni', 'dhi')):
    fig.line(irrad.index, irrad, legend=name, color=next(palette), line_width=2)
fig.xaxis.axis_label = 'Time (UTC)'
show(fig)

In [25]:
# returns tuple of ghi, dni, dhi, air temperature, wind speed, resampling function, solar position function
ghi, dni, dhi, air_temperature, wind_speed, resampler, solpos_func = models.hrrr_subhourly_to_subhourly_instantaneous(
    latitude, longitude, elevation, init_time, start, end, load_forecast=load_forecast)

Plot the irradiance components of `out`.

In [26]:
fig = figure(title="Irradiance (W/m^2)", tools=TOOLS, x_axis_type="datetime", plot_width=800, plot_height=400)
palette = iter(PALETTE)
for irrad, name in zip((ghi, dni, dhi), ('ghi', 'dni', 'dhi')):
    resampled = resampler(irrad)
    fig.line(resampled.index, resampled, legend=name, color=next(palette), line_width=2)
fig.xaxis.axis_label = 'Time (UTC)'
show(fig)

In [29]:
ghi, dni, dhi, air_temperature, wind_speed, resampler, solpos_func = models.gfs_quarter_deg_hourly_to_hourly_mean(
    latitude, longitude, elevation, init_time, start, end, load_forecast=load_forecast)

fig = figure(title="Irradiance (W/m^2)", tools=TOOLS, x_axis_type="datetime", plot_width=800, plot_height=400)
palette = iter(PALETTE)
for irrad, name in zip((ghi, dni, dhi), ('ghi', 'dni', 'dhi')):
    resampled = resampler(irrad)
    color = next(palette)
    fig.line(resampled.index, resampled, legend=name, color=color, line_width=2)
    fig.line(irrad.index, irrad, legend=name, color=color, line_width=2)
fig.xaxis.axis_label = 'Time (UTC)'
show(fig)

In [ ]:
forecast = datamodel.Forecast(
    name='NWP fx', 
    issue_time_of_day=datetime.time(0),  # 0Z
    lead_time_to_start=pd.Timedelta('1hr'),
    interval_length=pd.Timedelta('1hr'),
    run_length=pd.Timedelta('24hr'),
    interval_label='ending',
    interval_value_type='mean',
    variable='ghi',
    site=site
)

## Persistence forecasts

In [8]:
from solarforecastarbiter.reference_forecasts import persistence